## Import library

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from datetime import datetime
import requests

## Read covid_19_india.csv file

In [3]:
data = pd.read_csv("covid_19_india.csv")

data.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,2020-01-30,6:00 PM,Kerala,1,0,0,0,1
1,2,2020-01-31,6:00 PM,Kerala,1,0,0,0,1
2,3,2020-02-01,6:00 PM,Kerala,2,0,0,0,2
3,4,2020-02-02,6:00 PM,Kerala,3,0,0,0,3
4,5,2020-02-03,6:00 PM,Kerala,3,0,0,0,3


## Change columns Name

In [4]:
data = data.rename(columns={"State/UnionTerritory":"States","Cured":"Recovery"})

data.head()

,Sno,Date,Time,States,ConfirmedIndianNational,ConfirmedForeignNational,Recovery,Deaths,Confirmed
0,1,2020-01-30,6:00 PM,Kerala,1,0,0,0,1
1,2,2020-01-31,6:00 PM,Kerala,1,0,0,0,1
2,3,2020-02-01,6:00 PM,Kerala,2,0,0,0,2
3,4,2020-02-02,6:00 PM,Kerala,3,0,0,0,3
4,5,2020-02-03,6:00 PM,Kerala,3,0,0,0,3


## Remove Extra Columns

In [5]:
data = data.drop(['Sno','Time','ConfirmedIndianNational','ConfirmedForeignNational'],axis=1)

data.head()

,Date,States,Recovery,Deaths,Confirmed
0,2020-01-30,Kerala,0,0,1
1,2020-01-31,Kerala,0,0,1
2,2020-02-01,Kerala,0,0,2
3,2020-02-02,Kerala,0,0,3
4,2020-02-03,Kerala,0,0,3


## Get Active Cases

In [6]:
data['Active'] = data['Confirmed'] - data['Recovery'] - data['Deaths'] 

data.head()

,Date,States,Recovery,Deaths,Confirmed,Active
0,2020-01-30,Kerala,0,0,1,1
1,2020-01-31,Kerala,0,0,1,1
2,2020-02-01,Kerala,0,0,2,2
3,2020-02-02,Kerala,0,0,3,3
4,2020-02-03,Kerala,0,0,3,3


## Sort Values By Date and States

In [7]:
data = data.sort_values(['Date','States']).reset_index(drop=True)

data['Date'] = pd.to_datetime(data['Date'])

data

,Date,States,Recovery,Deaths,Confirmed,Active
0,2020-01-30,Kerala,0,0,1,1
1,2020-01-31,Kerala,0,0,1,1
2,2020-02-01,Kerala,0,0,2,2
3,2020-02-02,Kerala,0,0,3,3
4,2020-02-03,Kerala,0,0,3,3
...,...,...,...,...,...,...
16845,2021-07-07,Telangana,613124,3703,628282,11455
16846,2021-07-07,Tripura,63964,701,68612,3947
16847,2021-07-07,Uttar Pradesh,1682130,22656,1706818,2032
16848,2021-07-07,Uttarakhand,332006,7338,340882,1538


## Get total of all india Recovery, Deaths, Confirmed, Active cases

In [8]:
india_cases = data[data['Date'] == data['Date'].max()].copy().fillna(0)
india_cases.index = india_cases['States']
india_cases = india_cases.drop(['States','Date'],axis=1)

In [9]:
df = pd.DataFrame(pd.to_numeric(india_cases.sum()), dtype=int).transpose()
df.style.background_gradient(cmap='summer_r',axis=1)

,Recovery,Deaths,Confirmed,Active
0,29799534,404211,30663665,459920


## Get sorted values by Active cases

In [10]:
india_cases.sort_values('Active',ascending=False).style\
    .background_gradient(cmap='YlGn_r',subset=['Confirmed'])\
    .background_gradient(cmap='BrBG_r',subset=['Deaths'])\
    .background_gradient(cmap='BuPu',subset=['Recovery'])\
    .background_gradient(cmap='YlOrBr',subset=['Active'])

,Recovery,Deaths,Confirmed,Active
States,,,,
Maharashtra,5872268,123531,6113335,117536
Kerala,2877557,13960,2996094,104577
Karnataka,2784030,35526,2859595,40039
Tamil Nadu,2435872,33132,2503481,34477
Andhra Pradesh,1861937,12898,1908065,33230
Odisha,897362,4299,927186,25525
Assam,493306,4717,522267,24244
West Bengal,1472132,17834,1507241,17275
Telangana,613124,3703,628282,11455


## Imports For Charts

In [11]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot

Function for show charts

In [12]:
def horizontal_bar_chart(df,x,y,title,x_lable,y_lable,color):
    fig = px.bar(df, x=x, y=y, orientation='h', title=title, labels={x.name: x_lable,y.name:y_lable},color_discrete_sequence=[color])
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    fig.show()

In [13]:
cnf,dht,rec,act = '#393e46', '#33ccff', '#ff99cc', '#fe9801'

## Get Top 10 States with most confirmed cases

In [14]:
top_10_confirm_cases = india_cases.sort_values('Confirmed',ascending=False)[:10]
horizontal_bar_chart(top_10_confirm_cases,top_10_confirm_cases.Confirmed,top_10_confirm_cases.index,"Top 10 States with most confirmed cases",'Number of confirmed cases','State Name',cnf)

## Get Top 10 States with most Deaths cases

In [15]:
top_10_deaths_cases = india_cases.sort_values('Confirmed',ascending=False)[:10]
horizontal_bar_chart(top_10_deaths_cases,top_10_deaths_cases.Deaths,top_10_confirm_cases.index,"Top 10 States with most Deaths cases",'Number of Deaths cases','State Name',dht)

## Get Top 10 States with most Active cases

In [16]:
top_10_active_cases = india_cases.sort_values('Confirmed',ascending=False)[:10]
horizontal_bar_chart(top_10_active_cases,top_10_active_cases.Active,top_10_confirm_cases.index,"Top 10 States with most Active cases",'Number of Active cases','State Name',act)

## Get Top 10 States with most Recovery cases

In [17]:
top_10_recovery_cases = india_cases.sort_values('Confirmed',ascending=False)[:10]
horizontal_bar_chart(top_10_recovery_cases,top_10_recovery_cases.Recovery,top_10_confirm_cases.index,"Top 10 States with most Recovery cases",'Number of Recovery cases','State Name',rec)